<a href="https://colab.research.google.com/github/exphon/exphon2026/blob/main/Parselmouth_Tutorial_Part_II.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Parselmouth 튜토리얼 II


## 전제
- 로컬 컴퓨터의 `/data` 폴더에 아래 10개 파일이 있습니다.
  - `LJ001-0001.wav` ~ `LJ001-0005.wav`
  - `LJ001-0001.TextGrid` ~ `LJ001-0005.TextGrid`

- 이 `/data` 폴더를 **Google Drive**로 업로드합니다.

> 💡 진행 방법: 위에서 아래로 **셀을 순서대로 실행**하세요.  
> 에러가 나면: **바로 위 셀부터** 다시 실행하면 해결되는 경우가 많습니다.


## Step 0) 필요한 패키지 설치

In [3]:
!pip -q install praat-parselmouth
!pip -q install tgt pandas

### data 폴더 경로 지정

⚠️ 아래 `DATA_DIR`는 여러분이 Drive에 업로드한 위치에 맞게 수정해야 합니다.


In [4]:
# 터미널에서
# mkdir data
# mv *.wav data
# mv *.TextGrid data

## Step 1) 한 파일(phone 전부)을 표(DataFrame)로 만들기

In [5]:
import parselmouth
from parselmouth.praat import call
import glob, re, tgt

# 파일 하나만
for tg_file in glob.glob("data/*1.TextGrid"):
    print(f"Processing... {tg_file}")

    dir_name = tg_file.split("/")[:-1]
    tg_name = tg_file.split("/")[-1]
    wav_name = re.sub("TextGrid", "wav", tg_name)

    tg = tgt.read_textgrid(tg_file)
    print(tg.get_tier_names())

    phn_tier = tg.get_tier_by_name('phones')
    wrd_tier = tg.get_tier_by_name('words')

    overlap_tier = tgt.util.get_overlapping_intervals(phn_tier, wrd_tier)

    # Parselmouth에서 Praat을 통제하여 음성 자질 추출하기
    wave_full_name = str(dir_name[0]) + '/' + wav_name
    sound = parselmouth.Sound(wave_full_name)

    pitchobj = call(sound, "To Pitch", 0.01, 75, 300)

    print(wave_full_name)
    print('idx','\t','phn','\t','word','\t','start','\t','end','\t','dur')
    for i, t in enumerate(overlap_tier):
        # 처음 10개만
        if i > 10: break
        texts = t.text.split("+")
        phone = texts[0]
        word = texts[1]
        duration = (t.end_time-t.start_time)*1000  # milliseconds
        mid_time = t.start_time + (t.end_time - t.start_time)/2

        pitchvalue = call(pitchobj, "Get value at time", mid_time, "Hertz", "Linear")

        print(f"{i}",'\t',f"{phone}",'\t',f"{word}",'\t',f"{t.start_time}",'\t',f"{t.end_time}",'\t',f"{duration:.1f}",'\t',f"{pitchvalue:.1f}")
        #print(t.end_time)

        #print(texts[0])
        #print(texts[1])

Processing... data/LJ001-0001.TextGrid
['words', 'phones']
data/LJ001-0001.wav
idx 	 phn 	 word 	 start 	 end 	 dur
0 	 P 	 printing 	 0.0 	 0.04 	 40.0 	 nan
1 	 R 	 printing 	 0.04 	 0.07 	 30.0 	 203.8
2 	 IH1 	 printing 	 0.07 	 0.19 	 120.0 	 217.9
3 	 N 	 printing 	 0.19 	 0.22 	 30.0 	 211.0
4 	 T 	 printing 	 0.22 	 0.3 	 80.0 	 nan
5 	 IH0 	 printing 	 0.3 	 0.45 	 150.0 	 178.3
6 	 NG 	 printing 	 0.45 	 0.68 	 230.0 	 174.3
7 	 IH1 	 in 	 0.79 	 0.94 	 150.0 	 nan
8 	 N 	 in 	 0.94 	 1.0 	 60.0 	 189.7
9 	 DH 	 the 	 1.0 	 1.04 	 40.0 	 180.9
10 	 IY0 	 the 	 1.04 	 1.16 	 120.0 	 182.0


## STEP 2) Pandas DataFrame으로 전환

In [6]:
import parselmouth
from parselmouth.praat import call
import glob, re, tgt
import pandas as pd

# 파일 하나만
df = []
for tg_file in glob.glob("data/*1.TextGrid"):
    print(f"Processing... {tg_file}")

    dir_name = tg_file.split("/")[:-1]
    tg_name = tg_file.split("/")[-1]
    wav_name = re.sub("TextGrid", "wav", tg_name)

    tg = tgt.read_textgrid(tg_file)
    print(tg.get_tier_names())

    phn_tier = tg.get_tier_by_name('phones')
    wrd_tier = tg.get_tier_by_name('words')

    overlap_tier = tgt.util.get_overlapping_intervals(phn_tier, wrd_tier)

    # Parselmouth에서 Praat을 통제하여 음성 자질 추출하기
    wave_full_name = str(dir_name[0]) + '/' + wav_name
    sound = parselmouth.Sound(wave_full_name)

    pitchobj = call(sound, "To Pitch", 0.01, 75, 300)

    print(wave_full_name)
    rows = []
    print('idx','\t','phn','\t','word','\t','start','\t','end','\t','dur')
    for i, t in enumerate(overlap_tier):
        # 처음 10개만
        if i > 10: break
        texts = t.text.split("+")
        phone = texts[0]
        word = texts[1]
        duration = (t.end_time-t.start_time)*1000  # milliseconds
        mid_time = t.start_time + (t.end_time - t.start_time)/2

        pitchvalue = call(pitchobj, "Get value at time", mid_time, "Hertz", "Linear")

        #print(f"{i}",'\t',f"{phone}",'\t',f"{word}",'\t',f"{t.start_time}",'\t',f"{t.end_time}",'\t',f"{duration:.1f}",'\t',f"{pitchvalue:.1f}")
        rows.append([i, phone, word, t.start_time, t.end_time, format(duration, ".1f"), format(pitchvalue, ".0f")])

df = pd.DataFrame(rows, columns=['index','phone','word', 'start', 'end','dur_ms','f0'])
df.head()


Processing... data/LJ001-0001.TextGrid
['words', 'phones']
data/LJ001-0001.wav
idx 	 phn 	 word 	 start 	 end 	 dur


,index,phone,word,start,end,dur_ms,f0
0,0,P,printing,0.00,0.04,40.0,nan
1,1,R,printing,0.04,0.07,30.0,204
2,2,IH1,printing,0.07,0.19,120.0,218
3,3,N,printing,0.19,0.22,30.0,211
4,4,T,printing,0.22,0.30,80.0,nan


## STEP 3) Pitch, Intensity, Formants at vowel midpoints

In [7]:
import parselmouth
from parselmouth.praat import call
import glob, re, tgt
import pandas as pd

# 파일 하나만
df = []
for tg_file in glob.glob("data/*1.TextGrid"):
    print(f"Processing... {tg_file}")

    dir_name = tg_file.split("/")[:-1]
    tg_name = tg_file.split("/")[-1]
    wav_name = re.sub("TextGrid", "wav", tg_name)

    tg = tgt.read_textgrid(tg_file)
    print(tg.get_tier_names())

    phn_tier = tg.get_tier_by_name('phones')
    wrd_tier = tg.get_tier_by_name('words')

    overlap_tier = tgt.util.get_overlapping_intervals(phn_tier, wrd_tier)

    # Parselmouth에서 Praat을 통제하여 음성 자질 추출하기
    wave_full_name = str(dir_name[0]) + '/' + wav_name
    sound = parselmouth.Sound(wave_full_name)

    pitchobj = call(sound, "To Pitch", 0.01, 75, 300)
    intensityobj = call(sound, "To Intensity", 100, 0, True)
    formantObj = call(sound, "To Formant (burg)", 0, 5, 5500, 0.005, 50)


    print(wave_full_name)
    rows = []
    print('idx','\t','phn','\t','word','\t','start','\t','end','\t','dur')
    for i, t in enumerate(overlap_tier):
        # 처음 10개만
        if i > 10: break
        texts = t.text.split("+")
        phone = texts[0]
        word = texts[1]
        duration = (t.end_time-t.start_time)*1000  # milliseconds
        mid_time = t.start_time + (t.end_time - t.start_time)/2

        pitchvalue = call(pitchobj, "Get value at time", mid_time, "Hertz", "Linear")

        #print(f"{i}",'\t',f"{phone}",'\t',f"{word}",'\t',f"{t.start_time}",'\t',f"{t.end_time}",'\t',f"{duration:.1f}",'\t',f"{pitchvalue:.1f}")
        rows.append([i, phone, word, t.start_time, t.end_time, format(duration, ".1f"), format(pitchvalue, ".0f")])

df = pd.DataFrame(rows, columns=['index','phone','word', 'start', 'end','dur_ms','f0'])
df.head()

Processing... data/LJ001-0001.TextGrid
['words', 'phones']
data/LJ001-0001.wav
idx 	 phn 	 word 	 start 	 end 	 dur


,index,phone,word,start,end,dur_ms,f0
0,0,P,printing,0.00,0.04,40.0,nan
1,1,R,printing,0.04,0.07,30.0,204
2,2,IH1,printing,0.07,0.19,120.0,218
3,3,N,printing,0.19,0.22,30.0,211
4,4,T,printing,0.22,0.30,80.0,nan


## Step 4) CSV 저장 (Drive로)

In [8]:
OUT_CSV = '/content/lj5_parselmouth_features.csv'
df.to_csv(OUT_CSV, index=False)
print('saved:', OUT_CSV)


saved: /content/lj5_parselmouth_features.csv


## Step 5) 5개 파일 batch: file별 phone feature 합치기

## Resources

- [Praat Scripting Tutorial by Tae-Jin Yoon](https://github.com/exphon/UKSK2022)
- [IMPRS-2022-Parselmouth-tutorial by Yannick Jadoul](https://github.com/YannickJadoul/IMPRS-2022-Parselmouth-tutorial/blob/main/Tutorial.ipynb)